In [1]:
import torch
from torch import nn
import numpy as np
torch.manual_seed(1)

In [3]:
print(torch.__version__)
torch.set_default_tensor_type('torch.FloatTensor')

1.9.0


# 3.3.1 生成数据集

In [4]:
num_inputs = 2
num_examples = 1000
true_w = [2,-3.4]
true_b = 4.2

features = torch.tensor(np.random.normal(0,1,(num_examples, num_inputs)),dtype=torch.float)#features 是1000×2的矩阵
labels = true_w[0] * features[:,0] + true_w[1]*features[:,1]+true_b #y= w1*x1 + w1*x2 +b
labels += torch.tensor(np.random.normal(0,0.01,size=labels.size()),dtype=torch.float32)#噪声项 服从均值为0,标准差为0.01的正太分布

# 3.3.2 读取数据

In [11]:
import torch.utils.data as Data

batch_size=10

#将训练数据的特征和标签组合
dataset = Data.TensorDataset(features,labels)

#把dataset 放入 DataLoader
data_iter = Data.DataLoader(
    dataset=dataset,#torch TensorDaraset format
    batch_size=batch_size,#mini batch size
    shuffle= True, #要不要打乱数据(打乱比较好)
    num_workers=2, #多线程来读数据
)

In [12]:
for X,y in data_iter:
    print(X,'\n',y)
    break

tensor([[-0.3230,  2.2911],
        [-0.3126, -0.2897],
        [ 1.0934, -1.5292],
        [ 0.1188,  0.6519],
        [ 0.5496,  0.0713],
        [ 0.3797, -1.8168],
        [-0.8118, -0.5444],
        [-0.2088,  0.1579],
        [-1.3958,  0.6291],
        [ 0.6445, -0.4987]]) 
 tensor([-4.2574,  4.5535, 11.5819,  2.2237,  5.0516, 11.1521,  4.4230,  3.2583,
        -0.7281,  7.2027])


# 3.3.3 定义模型

In [15]:
class LinearNet(nn.Module):
    def __init__(self,n_feature):
        super(LinearNet,self).__init__()
        self.linear = nn.Linear(n_feature,1)
    
    def forward(self,x):
        y=self.linear(x)
        return y

net = LinearNet(num_inputs)
print(net)#使用print 可以打印出网络的结构

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [21]:
#写法一
net = nn.Sequential(
    nn.Linear(num_inputs,1)
    # 此处还可以传入其他层
)

#写法二
net = nn.Sequential()
net.add_module('linear',nn.Linear(num_inputs,1))

#写法3
from collections import OrderedDict
net = nn.Sequential(OrderedDict([('linear',nn.Linear(num_inputs,1))]))

print(net)
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [22]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[ 0.6938, -0.2992]], requires_grad=True)
Parameter containing:
tensor([0.5303], requires_grad=True)


# 3.3.4 初始化模型参数

In [24]:
from torch.nn import init

init.normal_(net[0].weight, mean=0.0, std=0.01)#随机采样于均值为0,标准差为0.01的正态分布
init.constant_(net[0].bias, val=0.0)#也可以直接修改bias的data： net[0].bias.data.fill_(0)

Parameter containing:
tensor([0.], requires_grad=True)

In [25]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[0.0032, 0.0010]], requires_grad=True)
Parameter containing:
tensor([0.], requires_grad=True)


# 3.3.5 定义损失函数

In [26]:
loss=nn.MSELoss()

# 3.3.6 定义优化算法


In [27]:
import torch.optim as optim

opotimizer = optim.SGD(net.parameters(),lr=0.03)
print(opotimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [28]:
# 为不同子网络设置不同的学习率
# optimizer =optim.SGD([
#                 # 如果对某个参数不指定学习率，就使用最外层的默认学习率
#                 {'params': net.subnet1.parameters()}, # lr=0.03
#                 {'params': net.subnet2.parameters(), 'lr': 0.01}
#             ], lr=0.03)

In [29]:
# # 调整学习率
# for param_group in optimizer.param_groups:
#     param_group['lr'] *= 0.1 # 学习率为之前的0.1倍

# 3.3.7 训练模型

In [30]:
num_epochs = 3
for epoch in range(1,num_epochs + 1):
    for X,y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1,1))
        opotimizer.zero_grad()#梯度清零，等价于net.zero_grad()
        l.backward()
        opotimizer.step()
    print('epocj %d, loss: %f' % (epoch, l.item()))

epocj 1, loss: 0.000162
epocj 2, loss: 0.000073
epocj 3, loss: 0.000239


In [31]:
dense = net[0]
print(true_w, dense.weight.data)
print(true_b, dense.bias.data)

[2, -3.4] tensor([[ 2.0004, -3.4008]])
4.2 tensor([4.2002])
